In [1]:
import RIM
import torch
from torch import nn
import numpy as np
import pandas as pd
import os
import RIM_SequentialDataloader
from RIM import RIM
import faiss
import configparser
from torch.utils.data import DataLoader
from tqdm import tqdm
import csv

In [2]:
model = RIM(
    dataset_summary_file='./data/tmall/feateng_data/summary_dict.pkl',
    features=7, embed_size=16, dropout=0.3, hidden_layers=[200, 80, 1]
)
loaded_dict = torch.load('model.pth')
model.load_state_dict(loaded_dict)

<All keys matched successfully>

In [5]:
def process_data(dataset, model):
    prefix = './data/tmall/feateng_data/target'
    if dataset == 'train':
        df = pd.read_csv(os.path.join(prefix, 'target_train.csv'), header=None)
    elif dataset == 'test':
        df = pd.read_csv(os.path.join(prefix, 'target_test.csv'), header=None)
    else:
        df = pd.read_csv(os.path.join(prefix, 'search_pool.csv'), header=None)
    len = df.shape[0]
    print(len)
    batch_size = 500000
    
    with open(f'L:/5104/{dataset}_embed.csv', 'w', newline='') as output_file:
        writer = csv.writer(output_file)
        for tmp in range(0, len, 20000000):
            print(f'{tmp} / {len}')
            x, y = df.iloc[tmp:min(len, tmp+20000000), [0, 1, 2, 3, 4, 7, 8]].values, df.iloc[tmp:min(len, tmp+20000000), 9].values
            for i in tqdm(range(0, min(len, tmp+20000000), batch_size)):
                data = torch.tensor(x[i:min(i+batch_size, len), :])
                embed_data = model(data, use_embed=True).squeeze(1).detach().numpy()
                writer.writerows(embed_data)
                # break
    output_file.close()


In [75]:
process_data('test', model)

18376300


100%|██████████| 52/52 [16:12<00:00, 18.70s/it]


In [7]:
process_data('retrieval', model)

60426722
0 / 60426722


100%|██████████| 40/40 [17:07<00:00, 25.69s/it]


20000000 / 60426722


100%|██████████| 80/80 [17:03<00:00, 12.80s/it]


40000000 / 60426722


100%|██████████| 120/120 [17:05<00:00,  8.55s/it]


60000000 / 60426722


100%|██████████| 121/121 [00:22<00:00,  5.47it/s]
